<a href="https://colab.research.google.com/github/tazar09/h3_working_repo/blob/main/army_analyzer_10-feb2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import math, io, re, os
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict
from typing import Dict, List, Optional
from dataclasses import dataclass
from enum import Enum
from fpdf import FPDF
from datetime import datetime
from IPython.display import display, clear_output

# ============================================================================
# DATA MODELS & LOGIC
# ============================================================================
HERO_STAT_MULTIPLIER = 0.05

def canonical(n: str) -> str:
    """Standardizes strings: lowercase, no spaces, no special chars."""
    if not n or pd.isna(n): return ""
    return re.sub(r'[^a-z0-9]', '', str(n).lower())

class UnitType(Enum):
    INFANTRY = "Infantry"; SHOOTER = "Shooter"; FLYER = "Flyer"
    ARTILLERY = "Artillery"; UNKNOWN = "Unknown"

    @classmethod
    def from_str(cls, s):
        s = str(s).strip().capitalize()
        for member in cls:
            if member.value == s: return member
        return cls.UNKNOWN

@dataclass
class Creature:
    name: str; town: str; ai_value: int; fight_value: int; cost: int; unit_type: UnitType; speed: int; terrain: str; level: int

@dataclass
class HeroStats:
    player_name: str; hero_name: str; attack: int; defense: int
    @property
    def multiplier(self) -> float:
        return math.sqrt((1 + HERO_STAT_MULTIPLIER * self.attack) * (1 + HERO_STAT_MULTIPLIER * self.defense))

@dataclass
class ArmyAnalysis:
    hero: HeroStats; units: List[dict]; total_ai_scaled: float; total_fight_scaled: float; total_gold: int
    type_breakdown: Dict[str, float]; town_breakdown: Dict[str, float]
    avg_speed: float; top_3_avg_speed: float; tactical_factor: float; missing: List[str]

class CreatureDatabase:
    def __init__(self):
        self.creatures: Dict[str, Creature] = {}

    def load_from_file(self, content, filename):
        try:
            if filename.lower().endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
            else:
                df = pd.read_excel(io.BytesIO(content))

            df.columns = [canonical(c) for c in df.columns]

            col_map = {
                'creature': ['creature'], 'town': ['town'], 'ai': ['ai'],
                'fight': ['fight'], 'goldcost': ['goldcost', 'gold_cost'],
                'type': ['type'], 'speed': ['speed'], 'terrain': ['terrain'],
                'level': ['level', 'lvl']
            }

            final_map = {}
            for target, options in col_map.items():
                for col in df.columns:
                    if col in options: final_map[target] = col

            for col in final_map.values():
                if df[col].dtype in ['float64', 'int64']:
                    df[col] = df[col].fillna(0)
                else:
                    df[col] = df[col].fillna("")

            self.creatures = {}
            for _, row in df.iterrows():
                name_raw = str(row[final_map['creature']]).strip()
                if not name_raw: continue

                terrain_val = str(row[final_map['terrain']]).strip() if 'terrain' in final_map else ""
                level_val = int(float(row[final_map['level']])) if 'level' in final_map else 1

                self.creatures[canonical(name_raw)] = Creature(
                    name=name_raw,
                    town=str(row[final_map['town']]).strip(),
                    ai_value=int(float(row[final_map['ai']])),
                    fight_value=int(float(row[final_map['fight']])),
                    cost=int(float(row[final_map['goldcost']])),
                    unit_type=UnitType.from_str(row[final_map['type']]),
                    speed=int(float(row[final_map['speed']])),
                    terrain=terrain_val,
                    level=level_val
                )
            return f"Database Loaded: {len(self.creatures)} units found."
        except Exception as e:
            return f"Error loading database: {str(e)}"

    def get(self, name: str) -> Optional[Creature]:
        key = canonical(name)
        res = self.creatures.get(key)
        if not res and key.endswith('s'): res = self.creatures.get(key[:-1])
        return res

# ============================================================================
# ANALYSIS & PDF GENERATOR
# ============================================================================
class ArmyAnalyzer:
    def analyze(self, army_dict: Dict[str, int], hero: HeroStats, db: CreatureDatabase, battle_terrain: str, art_bonus: int) -> ArmyAnalysis:
        units_list, t_ai_base, t_fight_base, t_gold = [], 0.0, 0.0, 0
        type_ai, town_ai, missing = defaultdict(float), defaultdict(float), []
        mult = hero.multiplier

        for name, count in army_dict.items():
            c = db.get(name)
            if not c:
                missing.append(name)
                continue

            effective_speed = c.speed + art_bonus
            if battle_terrain != "None" and canonical(c.terrain) == canonical(battle_terrain):
                effective_speed += 1

            ai_v, fi_v, g_v = c.ai_value * count * mult, c.fight_value * count * mult, c.cost * count
            units_list.append({
                'name': c.name, 'count': count, 'ai': ai_v, 'fight': fi_v,
                'gold': g_v, 'speed': effective_speed, 'type': c.unit_type,
                'town': c.town, 'level': c.level
            })

            t_ai_base += c.ai_value * count
            t_fight_base += c.fight_value * count
            t_gold += g_v
            type_ai[c.unit_type.value] += ai_v
            town_ai[c.town] += ai_v

        scaled_ai, scaled_fight = t_ai_base * mult, t_fight_base * mult
        combat_units = [u for u in units_list if u['type'] != UnitType.ARTILLERY]
        speeds = [u['speed'] for u in combat_units]
        avg_s = sum(speeds) / len(speeds) if speeds else 0
        top_3 = sum(sorted(speeds, reverse=True)[:3]) / min(len(speeds), 3) if speeds else 0
        tact_f = scaled_ai / scaled_fight if scaled_fight > 0 else 0

        return ArmyAnalysis(hero, units_list, scaled_ai, scaled_fight, t_gold, dict(type_ai), dict(town_ai), avg_s, top_3, tact_f, missing)

class ReportGenerator:
    def __init__(self):
        self.buffer = []
        self.chart_paths = []

    def log(self, text, is_bold=False):
        print(text); self.buffer.append((text, is_bold))

    def _create_visuals(self, a1, a2):
        self.chart_paths = []
        sns.set_theme(style="whitegrid")

        # Radar Profiles
        categories = ['Mobility', 'Tactical Factor', 'Av. Level', 'Range Power', 'Town Unity', 'Flyer Power']

        # Battle Totals for Competitive Metrics
        total_battle_flyer_ai = a1.type_breakdown.get('Flyer', 0) + a2.type_breakdown.get('Flyer', 0)
        total_battle_range_ai = a1.type_breakdown.get('Shooter', 0) + a2.type_breakdown.get('Shooter', 0)
        total_battle_tactical = a1.tactical_factor + a2.tactical_factor

        def get_radar_values(a):
            mobility = a.avg_speed / 18

            # New Tactical Factor Calculation (Competitive Ratio)
            tact_ratio = (a.tactical_factor / total_battle_tactical) if total_battle_tactical > 0 else 0

            # Weighted Average Level
            weighted_lvl_sum = sum(u['level'] * u['ai'] for u in a.units)
            avg_lvl = (weighted_lvl_sum / a.total_ai_scaled) if a.total_ai_scaled > 0 else 0
            av_level_score = avg_lvl / 7.0

            # Range Power (Share of battle shooters)
            range_val = a.type_breakdown.get('Shooter', 0)
            range_p = (range_val / total_battle_range_ai) if total_battle_range_ai > 0 else 0

            # Town Unity Calculation
            unique_towns = len(set(u['town'] for u in a.units if u['town']))
            unity = (7 - unique_towns) / 7 if unique_towns > 0 else 0

            # Flyer Power (Share of battle flyers)
            fly_val = a.type_breakdown.get('Flyer', 0)
            flyer_p = (fly_val / total_battle_flyer_ai) if total_battle_flyer_ai > 0 else 0

            return [min(1.0, max(0, v)) for v in [mobility, tact_ratio, av_level_score, range_p, unity, flyer_p]]

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6), subplot_kw=dict(polar=True))
        angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
        angles += angles[:1]

        # Individual Radars
        for a, ax, color in [(a1, ax1, 'blue'), (a2, ax2, 'red')]:
            vals = get_radar_values(a); vals += vals[:1]
            ax.plot(angles, vals, color=color, linewidth=2)
            ax.fill(angles, vals, color=color, alpha=0.3)
            ax.set_xticks(angles[:-1]); ax.set_xticklabels(categories)
            ax.set_title(f"{a.hero.hero_name} Profile", pad=20)

        # Overlay Radar
        for a, color in [(a1, 'blue'), (a2, 'red')]:
            vals = get_radar_values(a); vals += vals[:1]
            ax3.plot(angles, vals, color=color, linewidth=2, label=a.hero.hero_name)
            ax3.fill(angles, vals, color=color, alpha=0.1)
        ax3.set_xticks(angles[:-1]); ax3.set_xticklabels(categories)
        ax3.set_title("Direct Comparison", pad=20)
        ax3.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

        plt.tight_layout()
        p1 = "radar_profile.png"; plt.savefig(p1, bbox_inches='tight'); self.chart_paths.append(p1); plt.close()

        # 2. Speed Hierarchy
        speed_data = []
        for a in [a1, a2]:
            for u in a.units:
                if u['type'] != UnitType.ARTILLERY:
                    speed_data.append({'Unit': u['name'], 'Speed': u['speed'], 'Owner': a.hero.hero_name})
        df_speed = pd.DataFrame(speed_data).sort_values('Speed', ascending=False)
        plt.figure(figsize=(10, 8))
        owners = df_speed['Owner'].unique()
        palette = sns.color_palette("tab10", len(owners))
        owner_color = {owner: palette[i] for i, owner in enumerate(owners)}
        bar_colors = [owner_color[o] for o in df_speed['Owner']]
        plt.barh(df_speed['Unit'], df_speed['Speed'], color=bar_colors, height=1.0, edgecolor='white', linewidth=0.3)
        plt.gca().invert_yaxis()
        plt.title("Battle Speed Hierarchy (Turn Order)")
        from matplotlib.patches import Patch
        legend_elements = [Patch(facecolor=owner_color[o], label=o) for o in owners]
        plt.legend(handles=legend_elements, title="Hero")
        p2 = "speed_chart.png"; plt.savefig(p2, bbox_inches='tight'); self.chart_paths.append(p2); plt.close()

        # 3. AI Pie Charts
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        for a, ax in [(a1, ax1), (a2, ax2)]:
            labels = [u['name'] for u in a.units]
            values = [u['ai'] for u in a.units]
            ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
            ax.set_title(f"{a.hero.hero_name}: AI Distribution")
        p3 = "pie_chart.png"; plt.savefig(p3, bbox_inches='tight'); self.chart_paths.append(p3); plt.close()

    def generate(self, a1: ArmyAnalysis, a2: ArmyAnalysis, meta: dict):
        self.buffer = []
        self._create_visuals(a1, a2)
        n1, n2 = a1.hero.hero_name, a2.hero.hero_name
        def p(v, tot): return (v / tot * 100) if tot > 0 else 0
        def d(v1, v2): return ((v1 / v2) - 1) * 100 if v2 > 0 else 0

        self.log(f"CHAMPIONSHIP: {meta['champ']} | GAME: {meta['game']} | DATE: {meta['time']}", True)
        self.log(f"BATTLE TERRAIN: {meta['terrain']}", True)
        self.log("="*85)

        for a in [a1, a2]:
            self.log(f"\n{a.hero.hero_name} Army Composition", True)
            self.log(f"{'Unit':<22} {'Lvl':<4} {'Count':<7} {'AI Value':<12} {'Fight Value':<12} {'Speed':<6}")
            self.log("-" * 70)
            for u in sorted(a.units, key=lambda x: x['ai'], reverse=True):
                self.log(f"{u['name']:<22} {u['level']:<4} {u['count']:<7,} {int(u['ai']):<12,} {int(u['fight']):<12,} {u['speed']:<6}")

        self.log("\n\n")
        self.log(f"Army Comparison", True)
        self.log(f"{'Metric':<30} {n1:<18} {n2:<18} {'Diff %':<10}")
        self.log("-" * 85)
        for lbl, v1, v2 in [("Total AI Strength", a1.total_ai_scaled, a2.total_ai_scaled),
                           ("Total Fight Strength", a1.total_fight_scaled, a2.total_fight_scaled),
                           ("Total Gold Cost", float(a1.total_gold), float(a2.total_gold)),
                           ("Average Army Speed", a1.avg_speed, a2.avg_speed),
                           ("Top 3 Units Avg Speed", a1.top_3_avg_speed, a2.top_3_avg_speed)]:
            self.log(f"{lbl:<30} {int(v1):<18,} {int(v2):<18,} {d(v1, v2):>+8.2f}%")

        self.log("\n\n")
        self.log("Breakdown by Unit Type (AI)", True)
        self.log(f"{'Category':<20} {n1+' Val':<12} {n1+' %':<8} {n2+' Val':<12} {n2+' %':<8} {'Diff %':<8}")
        self.log("-" * 85)
        for t in ["Flyer", "Infantry", "Shooter", "Artillery"]:
            v1, v2 = a1.type_breakdown.get(t, 0), a2.type_breakdown.get(t, 0)
            self.log(f"{t:<20} {int(v1):<12,} {p(v1, a1.total_ai_scaled):<8.2f} {int(v2):<12,} {p(v2, a2.total_ai_scaled):<8.2f} {d(v1, v2):>+8.2f}%")

        self.log("\n\n")
        self.log("Breakdown by Town (AI)", True)
        self.log(f"{'Category':<20} {n1+' Val':<12} {n1+' %':<8} {n2+' Val':<12} {n2+' %':<8}")
        self.log("-" * 85)
        all_towns = sorted(list(set(list(a1.town_breakdown.keys()) + list(a2.town_breakdown.keys()))))
        for t in all_towns:
            v1, v2 = a1.town_breakdown.get(t, 0), a2.town_breakdown.get(t, 0)
            self.log(f"{t:<20} {int(v1):<12,} {p(v1, a1.total_ai_scaled):<8.2f} {int(v2):<12,} {p(v2, a2.total_ai_scaled):<8.2f}")

        self.log("\n\n")
        self.log("--- Battle Move Order Simulation ---", True)
        self.log(f"{'Order':<6} {'Unit Name':<20} {'Owner':<12} {'Speed':<6}")
        self.log("-" * 45)
        all_move_units = []
        for a in [a1, a2]:
            for u in a.units:
                if u['type'] != UnitType.ARTILLERY:
                    all_move_units.append({'name': u['name'], 'owner': a.hero.hero_name, 'speed': u['speed']})
        all_move_units.sort(key=lambda x: x['speed'], reverse=True)
        for i, unit in enumerate(all_move_units, 1):
            self.log(f"{i:<6} {unit['name']:<20} {unit['owner']:<12} {unit['speed']:<6}")

        self.log("\n\n")
        self.log("DETAILED COMPARISON NOTES", True)
        self.log("="*25)
        self.log("* Power Balance:")
        ai_lead = n1 if a1.total_ai_scaled > a2.total_ai_scaled else n2
        fi_lead = n1 if a1.total_fight_scaled > a2.total_fight_scaled else n2
        self.log(f"{ai_lead} leads in AI Strength ({abs(d(a1.total_ai_scaled, a2.total_ai_scaled)):.2f}%).")
        self.log(f"{fi_lead} leads in Fight Strength ({abs(d(a1.total_fight_scaled, a2.total_fight_scaled)):.2f}%).")
        self.log(f"Tactical reliance factor for {n1} is ({a1.tactical_factor:.2f}) and for {n2} ({a2.tactical_factor:.2f}).")

        if all_move_units:
            self.log(f"\n* Mobility Dominance:")
            unique_speeds = sorted(list(set(u['speed'] for u in all_move_units)), reverse=True)
            s1 = unique_speeds[0]
            us1 = [u for u in all_move_units if u['speed'] == s1]
            self.log(f"{us1[0]['owner']} has the fastest unit and moves first with {', '.join([u['name'] for u in us1])} ({s1}).")
            spd_lead = n1 if a1.avg_speed > a2.avg_speed else n2
            self.log(f"{spd_lead} has higher average speed ({max(a1.avg_speed, a2.avg_speed):.2f}).")
            top3_lead = n1 if a1.top_3_avg_speed > a2.top_3_avg_speed else n2
            self.log(f"{top3_lead} has higher average speed among top 3 units ({max(a1.top_3_avg_speed, a2.top_3_avg_speed):.2f}).")

        self._save_pdf(a1, a2, meta)

    def _save_pdf(self, a1, a2, meta):
        pdf = FPDF(); pdf.add_page(); pdf.set_font("Courier", size=8)
        date_str = datetime.now().strftime("%d-%b%Y").lower()
        fname = f"{a1.hero.player_name}({a1.hero.hero_name})_{a2.hero.player_name}({a2.hero.hero_name})_game-{meta['game']}_champ-{meta['champ']}_{date_str}.pdf".replace(" ", "_")
        for line, is_bold in self.buffer:
            pdf.set_font("Courier", style=("B" if is_bold else ""), size=8)
            pdf.cell(0, 4.2, txt=line, ln=1)

        pdf.add_page()
        pdf.set_font("Courier", style="B", size=14)
        pdf.cell(0, 10, txt="VISUAL ANALYSIS & METRIC EXPLANATIONS", ln=1)
        y_cursor = 25
        for path in self.chart_paths:
            if y_cursor > 220:
                pdf.add_page(); y_cursor = 20
            img_w = 190 if "radar" in path else 180
            pdf.image(path, x=10, y=y_cursor, w=img_w)
            y_cursor += 90
            if "radar" in path:
                pdf.set_font("Courier", style="B", size=9)
                pdf.set_xy(10, y_cursor)
                explanations = [
                    "Mobility: Calculated from Average Army Speed (Scale 0-18).",
                    "Tactical Factor: Army Reliance / (Self + Opponent Total Reliance).",
                    "Av. Level: Weighted average level: Sum(Stack Level * Stack AI) / Total AI.",
                    "Range Power: Army Shooter AI / (Self + Opponent Total Shooter AI).",
                    "Town Unity: Morale focus calculated as (7 - Number of Towns) / 7.",
                    "Flyer Power: Army Flyer AI / (Self + Opponent Total Flyer AI)."
                ]
                for line in explanations:
                    pdf.cell(0, 4, txt=f"* {line}", ln=1)
                y_cursor += 25
        pdf.output(fname)
        for path in self.chart_paths:
            if os.path.exists(path): os.remove(path)
        print(f"\n>>> PDF Generated: {fname}")

# ============================================================================
# INTERFACE
# ============================================================================
def get_file_info(upload_val):
    if not upload_val: return None, None
    try:
        if isinstance(upload_val, (list, tuple)):
            f = upload_val[0]
            return f.get('name'), f.get('content')
        first_key = list(upload_val.keys())[0]
        f = upload_val[first_key]
        name = f.get('name') or (f.get('metadata', {}).get('name'))
        content = f.get('content')
        return name, content
    except: return None, None

db = CreatureDatabase()
db_up = widgets.FileUpload(description="1. Database")
db_st = widgets.Label(value="Status: Waiting...")
terrain_dd = widgets.Dropdown(
    options=["None", "Grass", "Snow", "Dirt", "Swamp", "Rough", "Subterranean", "Lava", "Sand", "Water"],
    value="None", description="Terrain:"
)

def on_db(change):
    name, content = get_file_info(db_up.value)
    if content: db_st.value = db.load_from_file(content, name)
db_up.observe(on_db, names='value')

def hero_box(hero, player):
    p = widgets.Dropdown(options=["Tuthankam0n", "Florynache", "Xtrreme", "Adyyy"], value=player, description='Player:')
    h = widgets.Text(description="Hero:", value=hero)
    at = widgets.IntSlider(description="Attack:", min=0, max=99, value=10)
    df = widgets.IntSlider(description="Defense:", min=0, max=99, value=10)
    art = widgets.Dropdown(options=[("+0", 0), ("+1", 1), ("+2", 2), ("+3", 3)], value=0, description='Speed Art:')
    up = widgets.FileUpload(description="2. Army .txt")
    tx = widgets.Textarea(description="List:", layout={'height': '180px'})
    def on_up(change):
        _, content = get_file_info(up.value)
        if content: tx.value = content.decode('utf-8', errors='ignore')
    up.observe(on_up, names='value')
    return widgets.VBox([p, h, at, df, art, up, tx]), p, h, at, df, art, tx

ui1, p1, h1, a1, d1, art1, t1 = hero_box("Rissa", "Xtrreme")
ui2, p2, h2, a2, d2, art2, t2 = hero_box("Pyre", "Adyyy")
btn = widgets.Button(description="GENERATE PDF REPORT", button_style='success', layout={'width': '98%'})
out = widgets.Output()

def run(b):
    with out:
        clear_output()
        if not db.creatures: print("ERROR: Upload Database first!"); return
        def parse(s):
            res = {}
            for line in s.splitlines():
                if ',' in line:
                    parts = line.split(',')
                    name = parts[0].strip(); count = re.sub(r'[^0-9]', '', parts[1])
                    if count: res[name] = int(count)
            return res
        meta = {"game": "2", "champ": "10", "time": datetime.now().strftime("%Y-%m-%d %H:%M"), "terrain": terrain_dd.value}
        an1 = ArmyAnalyzer().analyze(parse(t1.value), HeroStats(p1.value, h1.value, a1.value, d1.value), db, terrain_dd.value, art1.value)
        an2 = ArmyAnalyzer().analyze(parse(t2.value), HeroStats(p2.value, h2.value, a2.value, d2.value), db, terrain_dd.value, art2.value)
        ReportGenerator().generate(an1, an2, meta)

btn.on_click(run)
display(widgets.HTML("<h2>HoMM3 Report Generator + Visuals</h2>"), widgets.HBox([db_up, db_st, terrain_dd]), widgets.HBox([ui1, ui2]), btn, out)

HTML(value='<h2>HoMM3 Report Generator + Visuals</h2>')

Button(button_style='success', description='GENERATE PDF REPORT', layout=Layout(width='98%'), style=ButtonStyl…

Output()